In [4]:
import pandas as pd
import numpy as np
import os

In [2]:
file_path="/home/externalFolder/demo_final_item/all_data_1919/combine_all_chr/all_chr_combine_data.csv"

In [3]:
#===================================pandas读取==========================================================#

In [4]:
with open(file_path,"r") as df:
    type(df)

In [ ]:
input_data=pd.read_csv(file_path,header=None,dtype=np.float16,encoding="utf8")

In [ ]:
df=input_data.values

In [ ]:
df.head

In [ ]:
#===================================pandsas分块读取=====================================================#

In [13]:
input_data=pd.read_csv(file_path,header=None,dtype=str,encoding="utf8",chunksize=1000)

In [14]:
chunk_list=[]

In [15]:
for data_chunk in input_data:
    chunk_list.append(data_chunk)

In [16]:
chunk_list

[                                                     0
 0    0.00056603\t0.012313\t0.0010257999999999999\t0...
 1    0.0021316\t0.0027948\t0.0018874000000000002\t0...
 2    0.00047679\t0.0028969\t0.0015327\t0.00020075\t...
 3    0.010322\t0.18354\t0.012739\t0.00032566\t0.127...
 4    0.0052942\t0.004561399999999999\t0.00668080000...
 5    0.005329100000000001\t0.015158000000000001\t0....
 6    0.069825\t0.037018\t0.081963\t0.0023001\t0.186...
 7    0.0010700999999999998\t0.020619\t0.00252399999...
 8    0.0062776\t0.0028061\t0.008259899999999999\t0....
 9    0.00046416\t0.00021769\t0.00075317\t0.00025985...
 10   0.0045267\t0.084652\t0.011744\t0.0023066\t0.09...
 11   0.00026634\t0.00064609\t0.00037189\t0.00044176...
 12   0.00094341\t0.0016553000000000002\t0.0012518\t...
 13   0.0035559\t0.0015661\t0.0060483\t0.001902\t0.0...
 14   0.00023863\t0.0013275000000000001\t0.00054333\...
 15   0.0052688000000000006\t0.008135600000000002\t0...
 16   0.30661\t0.052378999999999995\t0.1398\t0.0

In [18]:
print(type(chunk_list),len(chunk_list))

<class 'list'> 21


In [21]:
#print(chunk_list[0],type(chunk_list[0]))

In [ ]:
#===================================numpy读取==========================================================#

In [ ]:
data = np.loadtxt(file_path,dtype=str,delimiter=",")

In [ ]:
data.head

In [ ]:
#===============================keras生成器分批读取====================================================#

In [21]:
path="/home/externalFolder/demo_final_item/all_data_1919/combine_all_chr/test50.csv"

In [33]:
'''
i=0
with open(path,"r") as data:
    print(type(data))
    for line in data:
        print(i,len(line),type(line),len(line.split()),type(line.split()))

        print("="*80)
        line = np.array(line.strip().split(),dtype=np.float16)
        print(line,line.shape,type(line))
        break
'''

'\ni=0\nwith open(path,"r") as data:\n    print(type(data))\n    for line in data:\n        print(i,len(line),type(line),len(line.split()),type(line.split()))\n\n        print("="*80)\n        line = np.array(line.strip().split(),dtype=np.float16)\n        print(line,line.shape,type(line))\n        break\n'

In [22]:
def generate_arrays_from_file(path,batch_size):
    while True:
        data = open(path)
        cnt = 0
        X =[]
        for line in data:
            x = np.array(line.strip().split(),dtype=np.float16)
            X.append(x)
            cnt += 1
            if cnt==batch_size:
                cnt = 0
                yield (np.array(X), np.array(X)) #返回一个数组，数组里面两个shape为(batch_size, 383800）的ndarry
                X = []
    f.close()

In [23]:
generator = generate_arrays_from_file(path,25)

In [5]:
valid_data_path="/home/externalFolder/demo_final_item/all_data_1919/combine_all_chr/valid_data.csv"

In [6]:
with open(valid_data_path) as valid_data_list:
    valid_data=[]
    for line in valid_data_list:
        line = np.array(line.strip().split(),dtype=np.float16)
        valid_data.append(line)
valid_data=np.array(valid_data)

In [7]:
type(valid_data)

numpy.ndarray

In [9]:
valid_data.shape

(777, 383800)

In [8]:
valid_data[:1]

array([[0.001158, 0.01101 , 0.00141 , ..., 0.      , 0.      , 0.      ]],
      dtype=float16)

In [10]:
from keras.layers import Input, Dense
from keras.models import Model

Using TensorFlow backend.


In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
encoding_dim = 100

In [13]:
input_img = Input(shape=(200*1919,))

In [14]:
encoded = Dense(encoding_dim, activation='relu')(input_img)

Instructions for updating:
Colocations handled automatically by placer.


In [28]:
decoded = Dense(200*1919, activation='relu')(encoded)

In [29]:
autoencoder = Model(input=input_img, output=decoded)

In [30]:
encoder = Model(input=input_img, output=encoded)

In [31]:
autoencoder.compile(optimizer='adam', loss='mse')

In [32]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 383800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               38380100  
_________________________________________________________________
dense_3 (Dense)              (None, 383800)            38763800  
Total params: 77,143,900
Trainable params: 77,143,900
Non-trainable params: 0
_________________________________________________________________


In [33]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 383800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               38380100  
Total params: 38,380,100
Trainable params: 38,380,100
Non-trainable params: 0
_________________________________________________________________


In [ ]:
autoencoder.fit(x_train, x_train,
                nb_epoch=50,
                batch_size=32,
                shuffle=True,
                validation_data=(x_test, x_test))

In [34]:
autoencoder.fit_generator(generator,
                    steps_per_epoch = 2,
                    epochs = 5,
                    verbose = 1,
                    validation_data = (valid_data,valid_data))

Epoch 1/5
2/2 [==============================] - 62s 31s/step - loss: 0.0014 - val_loss: 0.0016
Epoch 2/5
2/2 [==============================] - 45s 23s/step - loss: 0.0014 - val_loss: 0.0015
Epoch 3/5
1/2 [==============>...............] - ETA: 3s - loss: 0.0015

KeyboardInterrupt: 